## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [1]:
## Import all necessary libraries
import json
import boto3
import logging
from globals import *
from utils import load_config, read_from_s3

CONFIG_FILE=configs/config-llama2-70b-g5-p4d-trt.yml


In [2]:
## Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Load the config file to extract our endpoint.json file and its respective file path
config = load_config(CONFIG_FILE)

In [3]:
endpoint_info_str = read_from_s3(BUCKET_NAME, ENDPOINT_S3_PATH)
logger.info(f"endpoint from --> s3://{BUCKET_NAME}/{ENDPOINT_S3_PATH}")

# Process the retrieved content
if endpoint_info_str:
    try:
        endpoint_info_list = json.loads(endpoint_info_str)
        logger.info(f"Found information for {len(endpoint_info_list)} endpoints in the respective S3 path")
        logger.info(json.dumps(endpoint_info_list, indent=2))
    except json.JSONDecodeError as e:
        logger.error(f"Error parsing JSON from S3: {e}")
else:
    logger.error("Error reading from S3 or no data found")

[2024-01-29 12:41:09,315] p71945 {1472297992.py:2} INFO - endpoint from --> s3://fmbttest/data/models/llama2-70b-g5-p4d-trt-v1/endpoints.json
[2024-01-29 12:41:09,316] p71945 {1472297992.py:8} INFO - Found information for 2 endpoints in the respective S3 path
[2024-01-29 12:41:09,317] p71945 {1472297992.py:9} INFO - [
  {
    "experiment_name": "llama2-70b-g5.48xlarge-huggingface-pytorch-tgi-inference-2.0.1-tgi1.1.0",
    "endpoint": {
      "EndpointName": "llama-2-70b-g5-48xlarge-1706548144",
      "EndpointArn": "arn:aws:sagemaker:us-east-1:015469603702:endpoint/llama-2-70b-g5-48xlarge-1706548144",
      "EndpointConfigName": "llama-2-70b-g5-48xlarge-1706548144",
      "ProductionVariants": [
        {
          "VariantName": "AllTraffic",
          "DeployedImages": [
            {
              "SpecifiedImage": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04",
              "ResolvedImage": "763104351884.d

In [4]:
## initialize a sagemaker client
sm_client = boto3.client("sagemaker")
# Iterate over the endpoint_info_list and mark the items for deletion
for item in endpoint_info_list:   
    
    ## Extract the endpoint name from the deployed model configuration
    ep_name = item['endpoint']["EndpointName"]
    try:
        ## Describe the model endpoint 
        logger.info(f"Going to describing the endpoint -> {ep_name}")
        resp = sm_client.describe_endpoint(EndpointName=ep_name)
        
        ## If the given model endpoint is in service, delete it 
        if resp['EndpointStatus'] == 'InService':
            logger.info(f"going to delete {ep_name}")
            ## deleting the model endpoint
            sm_client.delete_endpoint(EndpointName=ep_name)
            logger.info(f"deleted {ep_name}")
    except Exception as e:
        logger.error(f"error deleting endpoint={ep_name}, exception={e}")

[2024-01-29 12:41:09,430] p71945 {707195403.py:10} INFO - Going to describing the endpoint -> llama-2-70b-g5-48xlarge-1706548144
[2024-01-29 12:41:09,693] p71945 {707195403.py:15} INFO - going to delete llama-2-70b-g5-48xlarge-1706548144
[2024-01-29 12:41:09,893] p71945 {707195403.py:18} INFO - deleted llama-2-70b-g5-48xlarge-1706548144
[2024-01-29 12:41:09,893] p71945 {707195403.py:10} INFO - Going to describing the endpoint -> llama2-70bdjl-2024-01-29-17-09-05-066-endpoint
[2024-01-29 12:41:09,998] p71945 {707195403.py:15} INFO - going to delete llama2-70bdjl-2024-01-29-17-09-05-066-endpoint
[2024-01-29 12:41:13,144] p71945 {707195403.py:18} INFO - deleted llama2-70bdjl-2024-01-29-17-09-05-066-endpoint
